### Logistic Regression

In [3]:
%run ml_helpers.ipynb
%run ml.ipynb

               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
838596   11241353     False  1232  2018     2     8   25    19   Third  0.02   
986579   11437366     False   714  2018     9    36    5    17   Third  0.21   
1141809  11658810     False  1821  2019     4    16   18     7   First  0.50   
1126156  11637015      True  2223  2019     3    13   26    17   Third  0.00   
1003402  11460074     False   726  2018     9    39   27    20   Third  0.00   
1166044  11694655     False   122  2019     5    21   21    13  Second  0.18   
1181795  11719258     False  1133  2019     6    24   10    11  Second  0.00   
822284   11218587     False  2511  2018     1     5   30    13  Second  0.00   
1260169  11833639     False  1223  2019     9    38   18     8  Second  0.00   
1314023  11916168     False  2232  2019    12    49    5     2   First  0.00   
975797   11423371     False  1021  2018     8    34   23    12  Second  0.00   
1084002  11571428     False  1821  2019 

Finished one-hot encoding...
Finished standardizing...
               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
1191547  11732670     False   533  2019     6    25   23     2   First  0.24   
879465   11296279      True  1531  2018     4    17   26     8  Second  0.00   
840170   11243106     False  1631  2018     2     9   28     8  Second  0.01   
1130357  11642299     False   321  2019     4    14    1    19   Third  0.00   
1265429  11840101     False  1233  2019     9    39   25     7   First  0.00   
1115452  11619313     False  1232  2019     3    10   10    21   Third  0.00   
1218156  11771663     False   522  2019     7    30   26     7   First  0.00   
1066156  11545398     False   923  2018    12    51   23    22   Third  0.00   
1187934  11726954     False  1112  2019     6    25   18    13  Second  0.00   
1327453  11931701     False   524  2019    12    52   24    19   Third  0.00   
1281637  11864139     False   611  2019    10    42   17    17   

Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
         Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  count_l_stops  \
1091580  1823  2019      2   Third  0.02   0.4    21     2            1.0   
1128424  1115  2019      3   First  0.40   0.0    44    31            0.0   
1249557  1731  2019      9   First  0.00   0.0    70    62            0.0   
1238108   924  2019      8   First  0.42   0.0    78    68            0.0   
1204106  1232  2019      7  Second  0.00   0.0    83    65            1.0   
...       ...   ...    ...     ...   ...   ...   ...   ...            ...   
1098113   935  2019      2   Third  0.20   0.1    32    29            1.0   
1184484  1123  2019      6   Third  0.04   0.0    69    52            1.0   
1123205  2422  2019      3  Second  0.00   0.0    47    31            1.0   
1185867  17

Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot 

In [4]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

In [7]:
#Read Data:
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=[beat,])
data = convert_to_categorical()

In [8]:
data.dtypes

Unnamed: 0               int64
beat                     int64
beat_num                 int64
district                 int64
sector                   int64
geometry                object
Year                     int64
Month                    int64
Watch                   object
Beat                     int64
Crimes                   int64
Arrest                   int64
Domestic                 int64
Serious                float64
count_l_stops          float64
count_bus_stops        float64
count_metra_stops      float64
count_restaurants      float64
count_bars             float64
count_daycares         float64
count_entertainment    float64
count_businesses       float64
road_distance_ft       float64
TOTAL POPULATION       float64
dist_to_police         float64
dist_to_hospital       float64
SNOW                   float64
TMAX                   float64
TMIN                   float64
high_crime             float64
high_crime_geog_lag    float64
dtype: object

In [5]:
models = {
    'LogisticRegression': LogisticRegression(), 
}
grid = {
    'LogisticRegression': [{'penalty': x, 'C': y, 'random_state': 0} 
                           for x in ('l2', 'none') \
                           for y in (0.01, 0.1, 1, 10, 100)]
}